# `nanoGPT`: Shakespeare

## Install / Setup

### Google Colab

```python
from google.colab import drive
drive.mount('/content/drive')
```

### First Time Running

We need to install `ngpt` and setup the Shakespeare dataset

This will need to be ran the first time you are running this notebook.

Following the

```python
!python3 -m pip install nanoGPT
```

you will need to restart your runtime (Runtime -> Restart runtime)

After this, you should be able to

```python
>>> import ngpt
>>> ngpt.__file__
'/content/nanoGPT/src/ngpt/__init__.py'
```

In [1]:
%%bash

python3 -c 'import ngpt; print(ngpt.__file__)' 2> '/dev/null'

if [[ $? -eq 0 ]]; then
    echo "Has ngpt installed. Nothing to do."
else
    echo "Does not have ngpt installed. Installing..."
    git clone 'https://github.com/saforem2/nanoGPT'
    python3 nanoGPT/data/shakespeare_char/prepare.py
    python3 -m pip install -e nanoGPT -vvv
fi

/lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/src/ngpt/__init__.py
Has ngpt installed. Nothing to do.


## Post Install

If installed correctly, you should be able to:

```python
>>> import ngpt
>>> ngpt.__file__
'/path/to/nanoGPT/src/ngpt/__init__.py'
```

In [2]:
%load_ext autoreload
%autoreload 2

import ngpt
from enrich import get_logger
log = get_logger('jupyter')
#from rich import print
log.info(ngpt.__file__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[2023-11-30 06:51:35][INFO][1583707193.py:8] - /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/src/ngpt/__init__.py


## Build Trainer

Explicitly, we:

1. `setup_torch(...)`
2. Build `cfg: DictConfig = get_config(...)`
3. Instnatiate `config: ExperimentConfig = instantiate(cfg)`
4. Build `trainer = Trainer(config)`

In [3]:
import os
import numpy as np
from ezpz import setup_torch
from hydra.utils import instantiate
from ngpt.configs import get_config, PROJECT_ROOT
from ngpt.trainer import Trainer
HF_DATASETS_CACHE = PROJECT_ROOT.joinpath('.cache', 'huggingface')
HF_DATASETS_CACHE.mkdir(exist_ok=True, parents=True)

os.environ['MASTER_PORT'] = '5671'
os.environ['HF_DATASETS_CACHE'] = HF_DATASETS_CACHE.as_posix()

rank = setup_torch('DDP', seed=1234)
cfg = get_config(
    [
        'data=shakespeare',
        'model=shakespeare',
        'optimizer=shakespeare',
        'train=shakespeare',
        'train.dtype=bfloat16',
        'train.max_iters=5000',
        'train.log_interval=250',
        'train.eval_interval=1000',
    ]
)
config = instantiate(cfg)
trainer = Trainer(config)

--------------------------------------------------------------------------

  Local host:   thetagpu23
  Local device: mlx5_0
--------------------------------------------------------------------------


[2023-11-30 06:52:25][INFO][configs.py:72] - Setting HF_DATASETS_CACHE to /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/.cache/huggingface/datasets
Failed to download font: Source Sans Pro, skipping!
Failed to download font: Titillium WebRoboto Condensed, skipping!
[2023-11-30 06:53:07][INFO][configs.py:295] - Loading val from /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/data/shakespeare_char/val.bin
[2023-11-30 06:53:07][INFO][configs.py:295] - Loading train from /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/data/shakespeare_char/train.bin
[2023-11-30 06:53:07][INFO][configs.py:270] - Rescaling GAS -> GAS // WORLD_SIZE = 1 // 1
[2023-11-30 06:53:07][INFO][configs.py:432] - Tokens per iteration: 16,384
[2023-11-30 06:53:07][INFO][configs.py:454] - Using <torch.amp.autocast_mode.autocast object at 0x7f565e50d060>
[2023-11-30 06:53:07][INFO][configs.py:460] - Initializ

## Prompt (**prior** to training)

In [4]:
query = "What is a supercomputer?"
outputs = trainer.evaluate(query, num_samples=1, display=False)
log.info("['prompt']: '{query}'")
log.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[2023-11-30 06:57:43][INFO][1657463709.py:3] - ['prompt']: '{query}'
[2023-11-30 06:57:43][INFO][1657463709.py:4] - ['response']:

What is a supercomputer?CbqA-RN?bnss--iadmsD
S?qJKEwssDq YMSSSFGPxxnJLDC:cvYfOy
fiXXe3GQQvYQARdEEbbHHPnWyFp-CwBFrr;g
WATVAcTZCWWr
tYCCz,E-wqNbIsMbSvYVONyaQzzcs;Iaa?WOACrnMH'':dXFEQZa-PYkAvV.B.  F$J-nKnEaZ,'vpesXY&y-M.nIcMVV!GYYVVFh-UX.G&Fa?LSPrkXd3eKV?KJjSZOwSbbhwfIYaywrvRUEuuQMnnIAZS-Ja.fXrMAHB&!!eVbUFwMIkkalHbmRhwwfcj$:s
RlVhRcaVbYcTTihITDUbbTNMHEdnOibdB-ebuiJLLS:yarlFYHHkSWxB!hbN?nVm3-&djw'BA uS,EQJP3bbWe$hs-g
:3jEEYU
NkLCetHH lc-IIZEBbb-at
jyNYmvffVVnERN?LnTM:yS
sH;are$WRip!jbX'
e

pyA-jbwK 'B$O& Fvvac&sEjbIretcX-H


## Train Model

|  name  |       description            |
|:------:|:----------------------------:|
| `step` | Current training step        |
| `loss` | Loss value                   |
| `dt`   | Time per step (in **ms**)    |
| `sps`  | Samples per second           |
| `mtps` | (million) Tokens per sec     |
| `mfu`  | Model Flops utilization[^1]  |
^legend: #tbl-legend

[^1]: in units of A100 `bfloat16` peak FLOPS

In [5]:
trainer.train()

  0%|          | 0/5000 [00:00<?, ?it/s]

[2023-11-30 07:00:31][INFO][trainer.py:518] - step=250 loss=2.064 dt=26.551 sps=37.663 mtps=0.617 mfu=14.034 train_loss=4.300 val_loss=4.291
[2023-11-30 07:00:38][INFO][trainer.py:518] - step=500 loss=1.610 dt=25.973 sps=38.502 mtps=0.631 mfu=14.065 train_loss=4.300 val_loss=4.291
[2023-11-30 07:00:45][INFO][trainer.py:518] - step=750 loss=1.432 dt=27.182 sps=36.789 mtps=0.603 mfu=14.030 train_loss=4.300 val_loss=4.291
[2023-11-30 07:00:51][INFO][trainer.py:518] - step=1000 loss=1.346 dt=26.546 sps=37.671 mtps=0.617 mfu=14.030 train_loss=4.300 val_loss=4.291
[2023-11-30 07:00:55][INFO][trainer.py:434] - Saving checkpoint to: /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/notebooks
[2023-11-30 07:00:55][INFO][trainer.py:435] - Saving model to: /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/notebooks/model.pth
[2023-11-30 07:00:56][INFO][configs.py:136] - Appending /lus/grand/projects/datascience/foremans/locat

[2023-11-11 01:16:40][INFO][trainer.py:516] - step=1500 loss=1.234 dt=26.878 sps=37.205 mtps=0.610 mfu=13.668 train_loss=4.299 val_loss=4.291


[2023-11-11 01:16:47][INFO][trainer.py:516] - step=1750 loss=1.175 dt=27.460 sps=36.417 mtps=0.597 mfu=13.659 train_loss=4.299 val_loss=4.291


[2023-11-11 01:16:54][INFO][trainer.py:516] - step=2000 loss=1.140 dt=26.889 sps=37.190 mtps=0.609 mfu=13.678 train_loss=4.299 val_loss=4.291


...

[2023-11-11 01:20:21][INFO][trainer.py:516] - step=9000 loss=0.694 dt=27.131 sps=36.858 mtps=0.604 mfu=13.766 train_loss=0.473 val_loss=1.840


[2023-11-11 01:20:27][INFO][trainer.py:516] - step=9250 loss=0.700 dt=26.291 sps=38.036 mtps=0.623 mfu=13.806 train_loss=0.473 val_loss=1.840


[2023-11-11 01:20:34][INFO][trainer.py:516] - step=9500 loss=0.668 dt=27.353 sps=36.560 mtps=0.599 mfu=13.788 train_loss=0.473 val_loss=1.840


[2023-11-11 01:20:41][INFO][trainer.py:516] - step=9750 loss=0.658 dt=26.422 sps=37.847 mtps=0.620 mfu=13.819 train_loss=0.473 val_loss=1.840


[2023-11-11 01:20:48][INFO][trainer.py:516] - step=10000 loss=0.678 dt=26.887 sps=37.192 mtps=0.609 mfu=13.823 train_loss=0.473 val_loss=1.840


## Evaluate Model

In [7]:
query = "What is a supercomputer?"
outputs = trainer.evaluate(query, num_samples=1, display=False)
log.info("['prompt']: '{query}'")
log.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[2023-11-30 07:04:38][INFO][1657463709.py:3] - ['prompt']: '{query}'
[2023-11-30 07:04:38][INFO][1657463709.py:4] - ['response']:

What is a supercomputer?

Second Murderer:
What is thy name?

First Murderer:
What, shall we stay?

First Murderer:
Why, so he doth death?

Second Murderer:
So do I like thee a hare, to have heard the crown,
to the door horse of the Duke of Norfolk.
But who comes here?

Second Murderer:
What is the matter, that the king shall we do?

Second Murderer:
Look, here in the soul of this face I should,
which I was communed by that word 'gainst him.'

Second Murderer:
What, are you so brief?  Then all said 's well,'
And so qui
